In [123]:
from datetime import datetime as dt

import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

now = dt.now()
day = now.strftime("%Y-%m-%d")

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession(".cache", expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 39.4739,
    "longitude": -0.3797,
    "hourly": "temperature_2m",
    "start_date": day,
    "end_date": day,
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left",
    )
}

hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data=hourly_data)

In [124]:
def get_temperature_data(now: dt) -> int:
    """
    Fetches temperature data for a specific day using the Open-Meteo API.

    Parameters:
        day (datetime): The date for which to fetch temperature data in 'YYYY-MM-DD' format.

    Returns:
        int: The temperature in degrees Celsius for the specified day at the current hour.
    """

    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession(".cache", expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": 39.4739,
        "longitude": -0.3797,
        "hourly": "temperature_2m",
        "start_date": day,
        "end_date": day,
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left",
        )
    }

    hourly_data["temperature_2m"] = hourly_temperature_2m

    hourly_dataframe = pd.DataFrame(data=hourly_data)

    now = pd.Timestamp(dt(now.year, now.month, now.day, now.hour, 0, 0), tz="UTC")

    temp = hourly_dataframe[hourly_dataframe["date"] == now]["temperature_2m"].values[0]

    return temp

In [125]:
hourly_dataframe

,date,temperature_2m
0,2025-06-18 00:00:00+00:00,24.419500
1,2025-06-18 01:00:00+00:00,24.169500
2,2025-06-18 02:00:00+00:00,24.019501
3,2025-06-18 03:00:00+00:00,23.669500
4,2025-06-18 04:00:00+00:00,23.369501
5,2025-06-18 05:00:00+00:00,22.969501
6,2025-06-18 06:00:00+00:00,23.669500
7,2025-06-18 07:00:00+00:00,24.819500
8,2025-06-18 08:00:00+00:00,26.319500
9,2025-06-18 09:00:00+00:00,27.869501


In [126]:
hourly_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   date            24 non-null     datetime64[ns, UTC]
 1   temperature_2m  24 non-null     float32            
dtypes: datetime64[ns, UTC](1), float32(1)
memory usage: 420.0 bytes


In [127]:
# hourly_dataframe["date"] = hourly_dataframe["date"].apply(lambda x: x.to_pydatetime())

In [128]:
now = pd.Timestamp(dt(now.year, now.month, now.day, now.hour, 0, 0), tz="UTC")
now

Timestamp('2025-06-18 20:00:00+0000', tz='UTC')

In [129]:
hourly_dataframe.date[0]

Timestamp('2025-06-18 00:00:00+0000', tz='UTC')

In [130]:
hourly_dataframe[hourly_dataframe["date"] == now]

,date,temperature_2m
20,2025-06-18 20:00:00+00:00,25.519501


In [131]:
from zoneinfo import ZoneInfo

In [132]:
dt.now(tz=ZoneInfo("UTC"))

datetime.datetime(2025, 6, 18, 18, 33, 5, 914449, tzinfo=zoneinfo.ZoneInfo(key='UTC'))

In [133]:
now = dt.now(tz=ZoneInfo("UTC"))

a = get_temperature_data(now)
a

np.float32(26.6695)

In [134]:
def get_range(temp):
    for i in range(0, 41, 5):
        if temp - 5 < i:
            return f"peso_{i}_{i + 5}"


get_range(a)

'peso_25_30'